In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import regex as re
import time

In [2]:
def counter(df, label):
  word_count = {}
  for cat in categories:
    word_dict = {}
    data = df[df[label] == cat]['headline'] +' '+ df[df[label] == cat]['short_description']
    for row in data:
      words = list(set(re.split("[^a-zA-Z]",row)))#.split(' ')
      for word in words:
        word = word.lower()
        if word in word_dict:
          word_dict[word] = word_dict[word]+1
        else:
          word_dict[word] = 1
    for word, count in dict(word_dict).items():
      if (len(word)<3) or (count<4):
        word_dict.pop(word)
    word_count[cat] = word_dict
  return pd.DataFrame(word_count).fillna(0)

In [3]:
def prob_cond(word, category, word_df, a=0):
  """word: word whose probability needs to be found
     category: prior condition of the probabilty
     word_df: dictionary of the count of word in categories
     a: laplace smoothing factor"""
  word = word.lower()
  category = category.upper()
  if word in vocab:
    prob = (word_df.loc[word,category]+a)/(word_df[category].sum()+a*len(word_df[category]))
  else:
    prob = (a)/(word_df[category].sum()+a*len(word_df[category]))
  return prob

In [4]:
def prob(word, prob_df):
  word = word.lower()
  probblty = 0
  try:
    for cat in prob_df.columns:
      probblty += prob_df.loc[word,cat]*p_cat[cat]
    return probblty
  except:
    return probblty


In [5]:
def train(df,label='category', smoothing = 1):
  global categories
  categories = df[label].unique()
  temp = df[label].value_counts(sort=False)/df[label].value_counts().sum()
  p_cat = temp.to_dict()

  word_count = counter(df, label)
  global vocab
  vocab = word_count.index
  pc_word = {}
  for cat in categories:
    word_prob = {}
    for word in vocab:#word_count[cat].keys():
      word_prob[word] = prob_cond(word,cat,word_count,a=smoothing)
    pc_word[cat] = word_prob
  pc_word = pd.DataFrame(pc_word)
  return p_cat, word_count, pc_word

In [6]:
def predict(X):
  cat_pred = np.array([])
  #data = X#df_val['headline']#df_val['short_description']+' '+df_val['headline']
  for row in X:
    val_words = list(set(re.split("[^a-zA-Z]",row)))
    for word in val_words:
      if (len(word)<3):
        val_words.remove(word)
    cat_prob = np.array([])
    for cat in categories:
      list_prob = np.array([])
      for word in val_words:
        if word in vocab:#try:
          list_prob = np.append(list_prob,pc_word.loc[word,cat])#,word_count,a=1))
        else:
          list_prob = np.append(list_prob,prob_cond(word,cat,word_count,a=smoothing))
      cat_prob = np.append(cat_prob,np.prod(list_prob)*p_cat[cat])
    cat_prob = cat_prob/sum(cat_prob)
    # id = np.argmax(cat_prob)
    cat_pred = np.append(cat_pred,np.argmax(cat_prob))
  return cat_pred

In [7]:
def accuracy(prediction, actual):
  accu = 0
  for i in range(len(prediction)):
    if prediction[i] == actual[i]:
      accu += 1
  accu = accu/len(prediction)
  return accu

In [8]:
!wget -O news.json 'https://drive.google.com/uc?export=download&id=1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp'
# # https://drive.google.com/file/d/1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp/view?usp=sharing

--2022-04-15 03:56:56--  https://drive.google.com/uc?export=download&id=1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp
Resolving drive.google.com (drive.google.com)... 172.253.63.113, 172.253.63.139, 172.253.63.101, ...
Connecting to drive.google.com (drive.google.com)|172.253.63.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-80-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6m1t7n8sfh45svi4e5sop8q4cnb19jfd/1649994975000/16694202268609887225/*/1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp?e=download [following]
--2022-04-15 03:56:59--  https://doc-10-80-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6m1t7n8sfh45svi4e5sop8q4cnb19jfd/1649994975000/16694202268609887225/*/1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp?e=download
Resolving doc-10-80-docs.googleusercontent.com (doc-10-80-docs.googleusercontent.com)... 142.250.188.193, 2607:f8b0:4004:836::2001
Connecting to doc-10-80-docs.googleusercontent.com (doc-10-80-d

In [9]:
df = pd.read_json('news.json',lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [10]:
len(df.category.unique())

41

In [11]:
df.category.unique()

array(['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
       'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES',
       'SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'RELIGION',
       'SCIENCE', 'LATINO VOICES', 'EDUCATION', 'COLLEGE', 'PARENTS',
       'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE', 'HEALTHY LIVING',
       'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST', 'FIFTY', 'ARTS',
       'WELLNESS', 'PARENTING', 'HOME & LIVING', 'STYLE & BEAUTY',
       'DIVORCE', 'WEDDINGS', 'FOOD & DRINK', 'MONEY', 'ENVIRONMENT',
       'CULTURE & ARTS'], dtype=object)

In [12]:
df['category'] = df['category'].replace(['CULTURE & ARTS','THE WORLDPOST','PARENTS','ARTS','STYLE','TASTE','WELLNESS','WORLDPOST'],
                                        ['ARTS & CULTURE','WORLD NEWS','PARENTING','ARTS & CULTURE','STYLE & BEAUTY','FOOD & DRINK','HEALTHY LIVING','WORLD NEWS'])

In [13]:
len(df.category.unique())

33

In [14]:
df.category.unique()

array(['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
       'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES',
       'SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'RELIGION',
       'SCIENCE', 'LATINO VOICES', 'EDUCATION', 'COLLEGE', 'PARENTING',
       'ARTS & CULTURE', 'STYLE & BEAUTY', 'GREEN', 'FOOD & DRINK',
       'HEALTHY LIVING', 'GOOD NEWS', 'FIFTY', 'HOME & LIVING', 'DIVORCE',
       'WEDDINGS', 'MONEY', 'ENVIRONMENT'], dtype=object)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category           200853 non-null  object        
 1   headline           200853 non-null  object        
 2   authors            200853 non-null  object        
 3   link               200853 non-null  object        
 4   short_description  200853 non-null  object        
 5   date               200853 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [16]:
df, df_test = train_test_split(df, test_size=0.1, stratify=df['category'])
# df_val, df_test = train_test_split(df_val, test_size=0.5, random_state=42)

In [17]:
smoothing = 1

In [18]:
# p_cat, word_count, pc_word = train(df, 'category', stpwrds=stpwrds, smoothing=smoothing)
# cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}


In [19]:
# start = time.time()
# input = df_val['headline']#+' '+df_val['short_description']
# predictions = predict(input)
# predictions = [cat_dict[v] for v in predictions]
# print(time.time()-start)

In [ ]:
cv= 5
score = []
for i in range(cv):
    df_train, df_val = train_test_split(df, test_size=0.1)#, random_state=42)
    p_cat, word_count, pc_word = train(df_train, 'category', smoothing=smoothing)
    cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}
    input = df_val['headline']#+' '+df_val['short_description']
    predictions = predict(input)
    predictions = [cat_dict[v] for v in predictions]
    actual = df_val['category'].values
    score.append(accuracy(predictions,actual))
np.mean(score)

In [27]:
p_cat, word_count, pc_word = train(df, 'category',smoothing=smoothing)
cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}
print("Probability of 'The' occuring in a headline is: ", round(prob('The',pc_word),3))
print("Probability of 'The' occuring in a 'Business' headline is: ", round(prob_cond('The'.lower(),'BUSINESS'.upper(),word_count, a=smoothing),3))

Probability of 'The' occuring in a headline is:  0.044
Probability of 'The' occuring in a 'Business' headline is:  0.045


In [28]:
input = df_test['headline']+' '+df_test['short_description']
predictions = predict(input)
predictions = [cat_dict[v] for v in predictions]
actual = df_test['category'].values
accuracy(predictions,actual)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


0.584237777556507

In [29]:
full_prob = []
for cat in categories:
  word_prob = []
  for word in vocab:
    word_prob.append(pc_word.loc[word,cat]*p_cat[cat]/prob(word,pc_word))
  full_prob.append(word_prob)
full_prob = np.array(full_prob)
full_prob_df = pd.DataFrame(full_prob.T,columns=categories,index=vocab)

top_word = []
for cat in categories:
  top_word.append(full_prob_df[cat].sort_values(ascending=False).index)
top_word = np.array(top_word)
top_words_df = pd.DataFrame(top_word.T,columns=categories)

In [30]:
top_words_df.iloc[:10,:16]

,WORLD NEWS,POLITICS,HEALTHY LIVING,BUSINESS,DIVORCE,HOME & LIVING,COMEDY,QUEER VOICES,PARENTING,GOOD NEWS,LATINO VOICES,ENTERTAINMENT,SCIENCE,SPORTS,IMPACT,MEDIA
0,rohingya,huffpollster,apnea,marketers,divorcing,homesubmissions,pranks,bisexual,darndest,prosthetic,latinas,gilmore,spacecraft,peyton,aspenideas,anchors
1,ceasefire,democrat,meditate,securities,alimony,repurposing,psychometer,homosexuality,nameberry,amputee,shakira,deadpool,astronomers,playoff,oxfam,shep
2,philippine,mcconnell,clinical,bernanke,divorces,decorations,dissects,grindr,potty,doggie,despacito,dwayne,telescope,redskins,mentoring,abramson
3,kabul,newsbrief,muscle,kalanick,finalized,appraises,flicks,heterosexual,breastfeed,paw,latinx,disick,asteroid,lakers,slum,rezaian
4,theresa,polling,insomnia,onboarding,divorc,patio,growz,gwist,dadmissions,doggone,alejandro,awakens,jupiter,manziel,fgm,bolling
5,besieged,kasich,cholesterol,geithner,cheater,wallpaper,kamau,laverne,diapers,retriever,leguizamo,guardians,pluto,finals,philanthropist,halperin
6,airstrike,dems,supplements,euros,uncoupling,renovation,carvey,visibility,daycare,chihuahua,latinxs,spielberg,meteor,mets,midwife,ramos
7,baghdad,progressives,superfoods,coo,divorcees,printable,stefon,doma,rosenkrantz,pooches,bilingual,hamill,comet,deflategate,rabies,newsweek
8,quake,scalia,acupuncture,wal,amicable,printables,klepper,glaad,nanny,bulldog,junot,flick,physics,goodell,livestream,zucker
9,cleric,pelosi,cardiovascular,libor,bethenny,adler,offerman,bisexuality,nickmom,deceased,rosario,tyga,particle,playoffs,mozambique,photojournalist


In [31]:
top_words_df.iloc[:10,16:]

,BLACK VOICES,FOOD & DRINK,CRIME,TRAVEL,ARTS & CULTURE,WEDDINGS,STYLE & BEAUTY,WOMEN,ENVIRONMENT,TECH,RELIGION,MONEY,GREEN,EDUCATION,WEIRD NEWS,COLLEGE,FIFTY
0,ish,baked,tsarnaev,traveler,imageblog,grooms,huffpoststyle,succinct,networx,android,bishops,prepaid,orca,repayment,fark,fraternities,boomer
1,hbcu,tastemakers,fugitive,resorts,illustrator,khalil,chanel,steinem,sloth,unconfirmed,catholics,overdraft,fisheries,pearson,ufo,princeton,lbl
2,coogler,savory,dzhokhar,attractions,picasso,kellee,denim,womanhood,mazzoni,ios,archbishop,deposit,orangutan,instructional,squirrel,hazing,ageism
3,blackvoices,flavored,affluenza,getaways,warhol,newlywed,lingerie,gretchen,giraffes,whatsapp,faiths,ripoff,orcas,esea,speeding,yak,perimenopause
4,blackness,ramen,hid,looney,playhouse,brides,upton,tampons,pandas,siri,theological,deductions,breeding,octogenarian,gopro,yik,midlifers
5,souleo,pudding,arias,amenities,gogh,theknot,fashionista,kits,polluting,password,synagogue,coupons,divestment,readiness,redditors,sae,grandparenting
6,negro,strawberry,vanishes,itinerary,theatrical,hitched,malfunctions,mcgorry,mclendon,nokia,nuns,foreclosures,habitat,nclb,farting,wesleyan,uncles
7,stephon,pancakes,aggravated,tripadvisor,basel,officiant,manicure,disempowered,rhinos,sensors,prophet,borrower,emaciated,charter,feline,sororities,nana
8,belafonte,shrimp,retrial,landmarks,artworks,bridezilla,dior,vaginas,algae,machinations,nun,budgeting,conservationists,refinancing,tortoise,sigma,seeker
9,emmett,tacos,jodi,carnival,kahlo,rsvp,pippa,childfree,aquarium,curation,episcopal,frugal,grizzly,assessments,ballsy,sleeprevolution,grandkids
